In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
#%%
import numpy as np 
import pandas as pd 

import sys
import pandas
import numpy
import scipy as sp
import scipy
import pdb

import curvefit
from curvefit.core.model import CurveModel



In [58]:
# model for the mean of the data
def generalized_error_function(t, params) :
    alpha = params[0]
    beta  = params[1]
    p     = params[2]
    return 0.5 * p * ( 1.0 + scipy.special.erf( alpha * ( t - beta ) ) )
#
# link function used for beta
def identity_fun(x) :
    return x
#
# link function used for alpha, p
def exp_fun(x) :
    return numpy.exp(x)
#
# inverse of function used for alpha, p
def log_fun(x) :
    return numpy.log(x)

In [65]:


#   %% china death rate
# china_death_rate = pd.read_csv('G:/My Drive/Dev/covid19/death_rate/covid19_china_death_rate.csv')
china_death_rate = pd.read_csv('G:/My Drive/Dev/covid19/death_rate/covid19_china_death_v02.csv')
china_death_rate['death_rate'] = china_death_rate['death_rate']/np.max(china_death_rate['death_rate'])
china_death_rate['death_rate_std'] = 0.01

china_death_rate.head()
np.max(china_death_rate['death_rate'])
china_death_rate.columns


hubei = china_death_rate.query('(province in "Hubei")')

,province,Country/Region,Lat,Long,date,value,population,death_rate,day,social_dist,one,death_rate_std
0,Anhui,China,31.8257,117.2264,2020-02-10,3,59500510,0.000640,0,0,1,0.01
1,Anhui,China,31.8257,117.2264,2020-02-11,4,59500510,0.000853,1,0,1,0.01
2,Anhui,China,31.8257,117.2264,2020-02-12,4,59500510,0.000853,2,0,1,0.01
3,Anhui,China,31.8257,117.2264,2020-02-13,5,59500510,0.001066,3,0,1,0.01
4,Anhui,China,31.8257,117.2264,2020-02-14,6,59500510,0.001279,4,0,1,0.01


1.0

Index(['province', 'Country/Region', 'Lat', 'Long', 'date', 'value',
       'population', 'death_rate', 'day', 'social_dist', 'one',
       'death_rate_std'],
      dtype='object')

In [66]:
# ------------------------------------------------------------------------
#%% curve_model - random effect + covarite model
num_fe = 4
data_frame   = china_death_rate
col_t        = 'day'
col_obs      = 'death_rate'
col_covs     = [ ['one'], ['one', 'social_dist'], ['one'] ]
col_group    = 'province'
param_names  = [ 'alpha', 'beta',       'p'     ]
link_fun     = [ exp_fun, identity_fun, exp_fun ]
var_link_fun = num_fe * [ identity_fun ]
fun          = generalized_error_function
col_obs_se   = 'death_rate_std'
#
curve_model = curvefit.core.model.CurveModel(
    data_frame,
    col_t,
    col_obs,
    col_covs,
    col_group,
    param_names,
    link_fun,
    var_link_fun,
    fun
)
self = curve_model

In [78]:



# -------------------------------------------------------------------------
#%% fit_params
#
num_group = len(np.unique(china_death_rate['province']))
# fe_init   = fe_true / 3.0
fe_init = np.array([1,15,0.2,3],dtype = "float") # for alpha, beta, social_distance_fator, p


# re_init   = numpy.zeros( num_fe )
num_fe = len(fe_init)
re_init = np.ones(num_fe * num_group)
fe_bounds = [ [-numpy.inf, numpy.inf] ] * num_fe
# re_bounds = [ [0.0, 0.0] ] * num_fe
re_bounds = [[-np.inf, np.inf]] * num_fe
options   = {
    'ftol' : 1e-12,
    'gtol' : 1e-12,
}

# -------------------------------------------------------------------------



In [79]:
fe_init

array([ 1. , 15. ,  0.2,  3. ])

In [80]:
#
curve_model.fit_params(
    fe_init,
    re_init,
    fe_bounds,
    re_bounds,
    options=options
)
fe_estimate = curve_model.result.x[:num_fe]
curve_model.result
fe_estimate

      fun: 1200.2533929937565
 hess_inv: <100x100 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.94068043e-05,  3.41196070e-04,  0.00000000e+00,  3.44815407e-04,
       -5.53664516e-06,  2.81934604e-05,  0.00000000e+00,  2.70092168e-04,
       -2.96296218e-07, -1.48812925e-05,  0.00000000e+00,  2.62600814e-05,
        6.26881355e-07, -6.09636498e-06,  0.00000000e+00, -3.24530489e-04,
       -7.38176046e-06,  4.14265833e-05,  0.00000000e+00, -1.95128961e-04,
        5.73454533e-06, -4.04113590e-05,  0.00000000e+00, -2.68816278e-04,
       -6.65576405e-06,  4.89833073e-05,  0.00000000e+00,  7.85405880e-05,
       -6.60797783e-06,  4.31677229e-05,  0.00000000e+00,  2.18822401e-04,
        1.36160770e-06, -2.16731993e-05,  0.00000000e+00, -3.42007656e-04,
       -7.11552893e-06,  3.88957806e-05,  0.00000000e+00, -1.56514496e-04,
        1.99983442e-05, -1.74364487e-04,  0.00000000e+00,  1.29267327e-03,
       -4.68306517e-06,  3.95863451e-05,  0.00000000e+00, -4.47292666e-04,

array([ 0.92652416, 14.88840913,  0.2       , -6.93587264])

In [73]:
self.df.head()

,province,Country/Region,Lat,Long,date,value,population,death_rate,day,social_dist,one,death_rate_std
0,Anhui,China,31.8257,117.2264,2020-02-10,3,59500510,0.000640,0,0,1,0.01
1,Anhui,China,31.8257,117.2264,2020-02-11,4,59500510,0.000853,1,0,1,0.01
2,Anhui,China,31.8257,117.2264,2020-02-12,4,59500510,0.000853,2,0,1,0.01
3,Anhui,China,31.8257,117.2264,2020-02-13,5,59500510,0.001066,3,0,1,0.01
4,Anhui,China,31.8257,117.2264,2020-02-14,6,59500510,0.001279,4,0,1,0.01


In [23]:
self.col_t

'day'

In [24]:
self.col_obs

'death_rate'

In [25]:
self.col_covs

[['one'], ['one', 'social_dist'], ['one']]

In [26]:
self.col_group

'state'

In [27]:
self.param_names

array(['alpha', 'beta', 'p'], dtype='<U5')

In [28]:
self.link_fun

[<function __main__.exp_fun(x)>,
 <function __main__.identity_fun(x)>,
 <function __main__.exp_fun(x)>]

In [29]:
self.var_link_fun

[<function __main__.identity_fun(x)>,
 <function __main__.identity_fun(x)>,
 <function __main__.identity_fun(x)>,
 <function __main__.identity_fun(x)>]

In [31]:
??self.fun

Signature: self.fun(t, params)
Docstring: <no docstring>
Source:   
def generalized_error_function(t, params) :
    alpha = params[0]
    beta  = params[1]
    p     = params[2]
    return 0.5 * p * ( 1.0 + scipy.special.erf( alpha * ( t - beta ) ) )
File:      c:\rhuang\workspace\python\curvefit\example\<ipython-input-15-cf4c26df37aa>
Type:      function


In [33]:
??self.loss_fun

Signature: self.loss_fun(x)
Docstring: <no docstring>
Source:   
def normal_loss(x):
    return 0.5*sum(x**2)
File:      c:\rhuang\workspace\python\curvefit\env\lib\site-packages\curvefit\core\functions.py
Type:      function


In [34]:
self.col_obs_se

In [35]:
col_obs_se

'death_rate_std'

In [37]:
len(self.group_names)

5

In [38]:
self.num_obs

294

In [39]:
self.num_params

3

In [40]:
self.num_groups

5

In [46]:
self.col_group, self.col_t

('state', 'day')

In [48]:
self.obs_se[:5]

array([0.20491953, 0.20491953, 0.20491953, 0.20491953, 0.20491953])

In [49]:
col_obs_se

'death_rate_std'

In [50]:
np.unique(self.obs_se)

array([0.20491953])

In [51]:
self.df.head()

,state,country,lat,long,date,value,population,lockdown,death_rate,day,social_dist,one
0,Beijing,China,40.1824,116.4142,2020-02-28,7,19612368,2020-01-23,0.006341,0,0,1
1,Beijing,China,40.1824,116.4142,2020-02-29,8,19612368,2020-01-23,0.007246,1,0,1
2,Beijing,China,40.1824,116.4142,2020-03-01,8,19612368,2020-01-23,0.007246,2,0,1
3,Beijing,China,40.1824,116.4142,2020-03-02,8,19612368,2020-01-23,0.007246,3,0,1
4,Beijing,China,40.1824,116.4142,2020-03-03,8,19612368,2020-01-23,0.007246,4,0,1


In [57]:
col_obs_se

'death_rate_std'

In [56]:
self.df[col_obs_se].values

KeyError: 'death_rate_std'

self.num_params

In [ ]:



# sort the dataframe by group
self.df.sort_values([self.col_group, self.col_t], inplace=True)

# extracting information
 = self.df[self.col_obs].values
self.obs_se = np.ones(self.num_obs) if self.col_obs_se is None else \
    self.df[col_obs_se].values

self.scale_obs_se = scale_obs_se
if self.scale_obs_se:
    self.obs_se *= np.abs(self.obs).mean()/self.obs_se.mean()

self.t = self.df[self.col_t].values
self.group = self.df[self.col_group].values
self.covs = [
    df[name].values
    for name in self.col_covs
]
self.fe_sizes = np.array([
    cov.shape[1]
    for cov in self.covs
])
self.fe_idx = utils.sizes_to_indices(self.fe_sizes)
self.num_fe = self.fe_sizes.sum()
self.num_re = self.num_groups*self.num_fe

# parameter information
self.param_idx = {
    name: i
    for i, name in enumerate(self.param_names)
}

# group information
self.group_sizes = {
    name: np.sum(self.group == name)
    for name in self.group_names
}
self.order_group_sizes = np.array([
    self.group_sizes[name]
    for name in self.group_names
])
self.order_group_idx = np.cumsum(self.order_group_sizes) - 1
group_idx = utils.sizes_to_indices(np.array([
    self.group_sizes[name]
    for name in self.group_names
]))
self.group_idx = {
    name: group_idx[i]
    for i, name in enumerate(self.group_names)
}

# place holder
self.param_shared = []
self.result = None
self.params = None
self.fe_gprior = np.array([[0.0, np.inf]]*self.num_fe)
self.re_gprior = np.array([[0.0, np.inf]]*self.num_fe)
self.fun_gprior = None

In [ ]:
# input data


def unzip_x(self, x):
    """Unzip raw input to fixed effects and random effects.

    Args:
        x (numpy.ndarray):
            Array contains all the fixed and random effects.

    Returns:
        fe (numpy.ndarray): fixed effects.
        re (numpy.ndarray): random effects.
    """
    fe = x[:self.num_fe]
    re = x[self.num_fe:].reshape(self.num_groups, self.num_fe)
    return fe, re

def compute_params(self, x, expand=True):
    """Compute parameters from raw vector.

    Args:
        x (numpy.ndarray):
            Array contains all the fixed and random effects.
        expand (bool, optional):
            If `expand` is `True`, then create parameters for every
            observation, else only create parameters for each group.

    Returns:
        params (numpy.ndarray):
            Array of parameters for the curve functional form, with shape
            (num_params, num_obs) or (num_params, num_groups).
    """
    fe, re = self.unzip_x(x)
    covs = self.covs
    if expand:
        re = np.repeat(re, self.order_group_sizes, axis=0)
    else:
        covs = [
            self.covs[i][self.order_group_idx, :]
            for i in range(len(self.covs))
        ]
    var = fe + re
    for i in range(self.num_fe):
        var[:, i] = self.var_link_fun[i](var[:, i])
    params = np.vstack([
        np.sum(cov*var[:, self.fe_idx[i]], axis=1)
        for i, cov in enumerate(covs)
    ])

    for i in range(self.num_params):
        params[i] = self.link_fun[i](params[i])

    return params

def objective(self, x):
    """Objective function.

    Args:
        x (numpy.ndarray):
            Model parameters.

    Returns:
        float:
            Objective value.
    """
    fe, re = self.unzip_x(x)
    params = self.compute_params(x)
    residual = (self.obs - self.fun(self.t, params))/self.obs_se
    # val = 0.5*np.sum(residual**2)
    val = self.loss_fun(residual)
    # gprior from fixed effects
    val += 0.5*np.sum(
        (fe - self.fe_gprior.T[0])**2/self.fe_gprior.T[1]**2
    )
    # gprior from random effects
    val += 0.5*np.sum(
        (re - self.re_gprior.T[0])**2/self.re_gprior.T[1]**2
    )
    # other functional gprior
    if self.fun_gprior is not None:
        params = self.compute_params(x, expand=False)
        val += 0.5*np.sum(
            (self.fun_gprior[0](params) - self.fun_gprior[1][0])**2/
            self.fun_gprior[1][1]**2
        )
    return val

def gradient(self, x, eps=1e-16):
    """Gradient function.

    Args:
        x (numpy.ndarray):
            Model parameters.
        eps (float, optional):
            Tolerance for automatic differentiation.

    Returns:
        numpy.ndarray:
            Gradient w.r.t. the model parameters.
    """
    finfo = np.finfo(float)
    step  = finfo.tiny / finfo.eps
    x_c = x + 0j
    grad = np.zeros(x.size)
    for i in range(x.size):
        x_c[i] += step*1j
        grad[i] = self.objective(x_c).imag/step
        x_c[i] -= step*1j

    return grad

def fit_params(self,
                fe_init,
                re_init=None,
                fe_bounds=None,
                re_bounds=None,
                fe_gprior=None,
                re_gprior=None,
                fun_gprior=None,
                fixed_params=None,
                smart_initialize=False,
                fixed_params_initialize=None,
                options=None,
                smart_init_options=None):
    """Fit the parameters.

    Args:
        fe_init (numpy.ndarray):
            Initial value for the fixed effects.
        re_init (numpy.ndarray, optional):
            Initial value for the random effects.
        fe_bounds (list of lists, optional):
            Bounds for fixed effects.
        re_bounds (list of lists, optional):
            Bounds for random effects.
        fe_gprior (list of lists, optional):
            Gaussian prior for fixed effects.
        re_gprior (list of lists, optional):
            Gaussian prior for random effects.
        fun_gprior (list of lists, optional):
            Functional Gaussian prior.
        fixed_params (list{str}, optional):
            A list of parameter names that will be fixed at initial value.
        smart_initialize (bool, optional):
            Whether or not to initialize a model's fixed effects based
            on the average fixed effects across many individual models
            fit with the same settings and the random effects
            based on the fixed effects deviation from the average
            in the individual models
        fixed_params_initialize (list{str}, optional):
            A list of parameter names that will be fixed at initial value during the smart initialization.
            Will be ignored if smart_initialize = False and raise warning.
        options (dict, optional):
            Options for the optimizer.
        smart_init_options (dict, optional):
            Options for the inner model
    """
    assert len(fe_init) == self.num_fe
    if fe_bounds is None:
        fe_bounds = [[-np.inf, np.inf]]*self.num_fe
    if re_bounds is None:
        re_bounds = [[-np.inf, np.inf]]*self.num_fe
    assert len(fe_bounds) == self.num_fe
    assert len(re_bounds) == self.num_fe

    if fe_gprior is not None:
        assert len(fe_gprior) == self.num_fe
        self.fe_gprior = np.array(fe_gprior)
    if re_gprior is not None:
        assert len(re_gprior) == self.num_fe
        self.re_gprior = np.array(re_gprior)
    if re_init is None:
        re_init = np.zeros(self.num_re)

    if fun_gprior is not None:
        assert len(fun_gprior) == 2
        assert fun_gprior[1][1] > 0.0

    self.fun_gprior = fun_gprior

    if fixed_params_initialize is not None:
        if not smart_initialize:
            raise Warning(f"You passed in an initialization parameter "
                            f"fixed_params_initialize {fixed_params_initialize} "
                            f"but set smart_initialize=False. Will ignore fixed_params_initialize.")

    if smart_init_options is not None:
        if options is None:
            raise RuntimeError("Need to pass in options if you pass in smart init options.")

    if smart_initialize:
        smart_initialize_options = deepcopy(options)
        if smart_init_options is not None:
            smart_initialize_options.update(smart_init_options)
        if self.num_groups == 1:
            raise RuntimeError("Don't do initialization for models with only one group.")

        fe_dict = get_initial_params(
            groups=self.group_names,
            model=self,
            fit_arg_dict=dict(
                fe_init=fe_init,
                fe_bounds=fe_bounds,
                fe_gprior=fe_gprior,
                fixed_params=fixed_params_initialize,
                options=smart_initialize_options,
            )
        )
        fe_init, re_init = compute_starting_params(fe_dict)
        print(f"Overriding fe_init with {fe_init}.")
        print(f"Overriding re_init with {re_init}.")

    x0 = np.hstack([fe_init, re_init])
    if fe_bounds is None:
        fe_bounds = np.array([[-np.inf, np.inf]]*self.num_fe)
    if re_bounds is None:
        re_bounds = np.array([[-np.inf, np.inf]]*self.num_fe)

    fe_bounds = np.array(fe_bounds)
    re_bounds = np.array(re_bounds)

    if fixed_params is not None:
        for param in fixed_params:
            param_id = self.param_idx[param]
            fe_bounds[param_id] = x0[param_id, None]
            re_bounds[param_id] = 0.0

    re_bounds = np.repeat(re_bounds[None, :, :], self.num_groups, axis=0)
    bounds = np.vstack([fe_bounds,
                        re_bounds.reshape(self.num_re, 2)])

    result = minimize(
        fun=self.objective,
        x0=x0,
        jac=self.gradient,
        method='L-BFGS-B',
        bounds=bounds,
        options=options
    )

    self.result = result
    self.params = self.compute_params(self.result.x, expand=False)

def compute_rmse(self, x=None, use_obs_se=True):
    """Compute the Root Mean Squre Error.

    Args:
        x (numpy.ndarray | None, optional):
            Provided solution array, if None use the object solution.
        use_obs_se (bool, optional):
            If True include the observation standard deviation into the
            calculation.

    Returns:
        float: root mean square error.
    """
    if x is None:
        assert self.result is not None
        x = self.result.x

    params = self.compute_params(x)
    residual = self.obs - self.fun(self.t, params)

    if use_obs_se:
        return np.sqrt(np.sum(residual**2/self.obs_se**2)/
                        np.sum(1.0/self.obs_se**2))
    else:
        return np.sqrt(np.mean(residual**2))

def predict(self, t, group_name='all', prediction_functional_form=None):
    """Predict the observation by given independent variable and group name.

    Args:
        t (numpy.ndarray):
            Array of independent variable.
        group_name (dtype(group_names) | str, optional):
            If all will produce average curve and if specific group name
            will produce curve for the group.
        prediction_functional_form (function):
            One of the functions from curvefit.functions
            Needs to have the same parameters as self.fun

    Returns:
        numpy.ndarray:
            Array record the curve.
    """
    if group_name == 'all':
        params = self.params.mean(axis=1)
    else:
        params = self.params[:, np.where(self.group_names == group_name)[0][0]]

    if prediction_functional_form is None:
        fun = self.fun
    else:
        fun = prediction_functional_form

    return fun(t, params)

def estimate_obs_se(self, radius=3.0, se_floor=0.5):
    """Estimate the observation standard error.

    Args:
        radius (float, optional):
            Radius group to estimate standard error.
        se_floor (float, optional):
            When the standard error is low use this instead.

    Returns:
        numpy.ndarray:
            Vector that contains all the standard error for each
            observation.
    """
    residual = self.obs - self.fun(self.t, self.params)
    residual = [
        residual[self.group_idx[name]]
        for name in self.group_names
    ]
    obs_se = []
    for j, name in enumerate(self.group_names):
        sub_residual = residual[j]
        sub_t = self.t[self.group_idx[name]]
        sub_obs_se = np.zeros(self.group_sizes[name])
        for i in range(self.group_sizes[name]):
            lb = max(0, sub_t[i] - radius)
            ub = min(sub_t.max(), sub_t[i] + radius)
            lb_idx = np.arange(self.group_sizes[name])[sub_t >= lb][0]
            ub_idx = np.arange(self.group_sizes[name])[sub_t <= ub][-1]
            sub_obs_se[i] = max(se_floor,
                                np.std(sub_residual[lb_idx:ub_idx]))
        obs_se.append(sub_obs_se)
    return np.hstack(obs_se)

def get_self_model_kwargs(self):
    """
    Gets keyword arguments for a CurveModel
    based on this instance of the CurveModel

    Returns:
        (dict) kwargs for model from self
    """
    return dict(
        col_t=self.col_t,
        col_obs=self.col_obs,
        col_covs=self.col_covs,
        col_obs_se=self.col_obs_se,
        col_group=self.col_group,
        param_names=self.param_names,
        link_fun=self.link_fun,
        var_link_fun=self.var_link_fun,
        fun=self.fun
    )

def run_one_group_model(self, group, **fit_kwargs):
    """
    Run the exact model as self but instantiate it as a new model
    so that we can run it on a subset of the data defined by the group (no random effects).
    Used for smart initialization of fixed and random effects.

    Args:
        group: (str) the random effect group to include
        in this model.

    Returns:
        np.array of fixed effects for this single group
    """
    df_sub = self.df.loc[self.df[self.col_group] == group].copy()
    model_kwargs = self.get_self_model_kwargs()
    model = CurveModel(df=df_sub, **model_kwargs)
    fit_dict = deepcopy(fit_kwargs)
    fit_dict.update(dict(
        re_bounds=[[0.0, 0.0] for i in range(model.num_fe)],
        smart_initialize=False
    ))
    model.fit_params(**fit_dict)
    return model.result.x[:self.num_fe]
